In [1]:
from truncatedLaplace import TruncatedLaplace
from truncatedGaussian import TruncatedGaussian
from histogramEstimator import HistogramEstimator
import seaborn as sns
import pandas as pd
import json
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from IPython.display import clear_output

In [2]:
a, b = 0, 1

In [ ]:
laplace = TruncatedLaplace(a=a, b=b, scale=1)
e_laplace = HistogramEstimator(
    mechanism=laplace,
    a=a,
    b=b,
    C=1.58,
    D=3.16,
    epsilon=1,
    delta=0.95,
    gamma=0.2,
)

In [4]:
laplace_rdp = TruncatedLaplace(a=a, b=b, scale=1.5)
e_laplace_rdp = HistogramEstimator(
    mechanism=laplace_rdp,
    a=a,
    b=b,
    C=0.913,
    D=1.83,
    epsilon=0.143,
    delta=0.95,
    gamma=0.04,
    renyi=True,
)

In [ ]:
gaussian = TruncatedGaussian(a=a, b=b, scale=0.6)
e_gaussian = HistogramEstimator(
    mechanism=gaussian,
    a=a,
    b=b,
    C=1.49,
    D=1.62,
    epsilon=1.39,
    delta=0.95,
    gamma=0.4,
)

In [6]:
gaussian_rdp = TruncatedGaussian(a=a, b=b, scale=1.5)
e_gaussian_rdp = HistogramEstimator(
    mechanism=gaussian_rdp,
    a=a,
    b=b,
    C=0.38,
    D=0.23,
    epsilon=0.016,
    delta=0.95,
    gamma=0.04,
    renyi=True,
)

In [7]:
def plot_n_pr(e):
    df = []
    n_th = e.n
    n = 1
    while True:
        e.n = n
        valid = 0
        for run in (pbar := tqdm(range(1, 101))):
            estimate = e.estimate(0, 1)
            df.append([n, run, estimate])
            error = abs(estimate - e.epsilon)
            pbar.set_description(
                f"valid_ratio = {valid / run:.2g} (need > {e.delta}), n = {n:.2g}, error = {error:.2g} (need < {e.gamma:.2g})"
            )
            if error < e.gamma:
                valid += 1
        if valid / 100 >= e.delta:
            break
        n *= 2
    df = pd.DataFrame(df, columns=["n", "run", "epsilon"])
    sns.lineplot(
        data=df,
        x="n",
        y="epsilon",
        label=r"Estimated $\tilde\epsilon$ across 100 runs",
    )
    sns.scatterplot(data=df, x="n", y="epsilon", alpha=0.1)
    plt.axhspan(
        e.epsilon - e.gamma,
        e.epsilon + e.gamma,
        color="orange",
        alpha=0.3,
        label=r"Span $\epsilon \pm \gamma$",
    )
    plt.axhline(
        y=e.epsilon, color="red", linestyle="--", label=r"Actual $\epsilon$"
    )
    plt.axvline(x=e.n, color="k", linestyle="--")
    plt.annotate(
        "Practical n",
        xy=(e.n, plt.ylim()[0]),
        xytext=(-35, 15),
        textcoords="offset points",
        ha="center",
        va="top",
    )
    plt.axvline(x=n_th, color="k", linestyle="--")
    plt.annotate(
        "Theoretical n",
        xy=(n_th, plt.ylim()[0]),
        xytext=(-40, 15),
        textcoords="offset points",
        ha="center",
        va="top",
    )
    plt.ylabel(r"$\tilde\epsilon$")
    plt.xscale("log")
    mechanism = e.mechanism.__class__.__name__.replace("Truncated", "")
    title = "Truncated " + mechanism
    file = "results/" + mechanism.lower()
    if e.renyi:
        title += " LRDP"
        file += "_rdp"
    else:
        title += " LDP"
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    with open(file + ".json", "w") as f:
        json.dump(
            {
                "n_th": n_th,
                "n_pr": e.n,
                "epsilon": e.epsilon,
                "C": e.C,
                "D": e.D,
                "gamma": e.gamma,
                "delta": e.delta,
            },
            f,
            indent=4,
        )
    df.to_csv(file + ".csv", index=False)
    plt.savefig(file + ".png")
    plt.show()
    print(title, "done")

In [ ]:
plot_n_pr(e_laplace)
clear_output()
plot_n_pr(e_laplace_rdp)
clear_output()
plot_n_pr(e_gaussian)
clear_output()
plot_n_pr(e_gaussian_rdp)